In [1]:
# add other directories to the path
import sys, os
notebook_path = os.path.abspath('')
parent_dir = os.path.join(notebook_path, '..')  # This gets to CVEFinder
root_dir = os.path.join(parent_dir, '..')  # This gets to ZeroDayResearcher root
sys.path.append(root_dir)  # Add project root to path


In [2]:
import os

from server.gmail.auth import get_gmail_service

In [3]:
service = get_gmail_service()

In [4]:
count: int = 10
query: str = ""

# Build search query
search_params = {
    'userId': 'me',
    'maxResults': count,
    'labelIds': ['INBOX']
}
        
results = service.users().messages().list(**search_params).execute()
    

In [5]:
results

{'messages': [{'id': '197dbb0f63cac5c4', 'threadId': '197dbb0f63cac5c4'},
  {'id': '197db7967323dd16', 'threadId': '197dae51290290e2'},
  {'id': '197daf0de65a0511', 'threadId': '197daf0de65a0511'},
  {'id': '197dae51290290e2', 'threadId': '197dae51290290e2'},
  {'id': '197daaf593c03408', 'threadId': '197daaf593c03408'},
  {'id': '197daaa89b192122', 'threadId': '197daaa89b192122'},
  {'id': '197da9ed8c8e9706', 'threadId': '197da9ed8c8e9706'},
  {'id': '197da517169ad9e3', 'threadId': '197da517169ad9e3'},
  {'id': '197da4527ef25b24', 'threadId': '197da4527ef25b24'},
  {'id': '197da27c126f59a4', 'threadId': '197da27c126f59a4'}],
 'nextPageToken': '08045794094481276842',
 'resultSizeEstimate': 201}

In [6]:
messages = results.get('messages', [])

In [7]:
import pprint

In [10]:
for msg in messages:
    # Use format='metadata' to work with current scope
    msg_data = service.users().messages().get(
        userId='me', 
        id=msg['id'],
        format='metadata',
        metadataHeaders=['subject', 'from', 'date', 'to', 'cc']
    ).execute()

    # Extract headers for easier access
    headers = {h['name'].lower(): h['value'] for h in msg_data['payload'].get('headers', [])}
    
    # Create a cleaner message object
    clean_msg = {
        'id': msg['id'],
        'snippet': msg_data.get('snippet', ''),
        'subject': headers.get('subject', 'No Subject'),
        'from': headers.get('from', 'Unknown Sender'),
        'to': headers.get('to', ''),
        'cc': headers.get('cc', ''),
        'date': headers.get('date', ''),
        'thread_id': msg_data.get('threadId', '')
    }

    pprint.pprint(clean_msg)

{'cc': '',
 'date': 'Sat, 5 Jul 2025 20:15:15 +0300 (MSK)',
 'from': 'Codeforces@codeforces.com',
 'id': '197dbb0f63cac5c4',
 'snippet': '',
 'subject': 'EPIC Institute of Technology Round Summer 2025 (Codeforces Round '
            '1036, Div. 1 + Div. 2) (Rated for Everybody)',
 'thread_id': '197dbb0f63cac5c4',
 'to': 'automatefreely@gmail.com'}
{'cc': '',
 'date': 'Sat, 5 Jul 2025 16:44:14 +0000 (UTC)',
 'from': 'Deven Lunkad via LinkedIn <messaging-digest-noreply@linkedin.com>',
 'id': '197db7967323dd16',
 'snippet': '',
 'subject': 'Deven just messaged you',
 'thread_id': '197dae51290290e2',
 'to': 'Swaroop Dora <automatefreely@gmail.com>'}
{'cc': '',
 'date': 'Sat, 5 Jul 2025 16:44:14 +0000 (UTC)',
 'from': 'Deven Lunkad via LinkedIn <messaging-digest-noreply@linkedin.com>',
 'id': '197db7967323dd16',
 'snippet': '',
 'subject': 'Deven just messaged you',
 'thread_id': '197dae51290290e2',
 'to': 'Swaroop Dora <automatefreely@gmail.com>'}
{'cc': '',
 'date': 'Sat, 5 Jul 2025 14:15

In [ ]:
import base64
import pprint
from datetime import datetime

def decode_base64_data(data):
    """Decode base64 encoded data safely"""
    if not data:
        return ""
    try:
        # Add padding if needed
        missing_padding = len(data) % 4
        if missing_padding:
            data += '=' * (4 - missing_padding)
        return base64.urlsafe_b64decode(data).decode('utf-8')
    except Exception as e:
        print(f"Error decoding base64 data: {e}")
        return f"[Could not decode: {data[:50]}...]"

def extract_message_body(payload):
    """Extract message body from Gmail API payload"""
    body = ""
    
    if 'parts' in payload:
        # Multi-part message
        for part in payload['parts']:
            if part['mimeType'] == 'text/plain':
                if 'data' in part['body']:
                    body = decode_base64_data(part['body']['data'])
                    break
            elif part['mimeType'] == 'text/html' and not body:
                # Fall back to HTML if no plain text
                if 'data' in part['body']:
                    body = decode_base64_data(part['body']['data'])
    else:
        # Single part message
        if 'body' in payload and 'data' in payload['body']:
            body = decode_base64_data(payload['body']['data'])
    
    return body

def get_header_value(headers, name):
    """Get header value by name"""
    for header in headers:
        if header['name'].lower() == name.lower():
            return header['value']
    return ""

def format_message_display(msg_data):
    """Format message data for clean display (metadata only)"""
    headers = msg_data.get('payload', {}).get('headers', [])
    
    # Extract key information
    subject = get_header_value(headers, 'Subject')
    from_addr = get_header_value(headers, 'From')
    to_addr = get_header_value(headers, 'To')
    date = get_header_value(headers, 'Date')
    
    # Format output
    print("=" * 80)
    print(f"MESSAGE ID: {msg_data.get('id', 'N/A')}")
    print(f"SUBJECT: {subject}")
    print(f"FROM: {from_addr}")
    print(f"TO: {to_addr}")
    print(f"DATE: {date}")
    print(f"SNIPPET: {msg_data.get('snippet', 'N/A')}")
    print("=" * 80)
    print()

# Your improved message processing loop (metadata scope)
for msg in messages:
    msg_data = service.users().messages().get(
        userId='me', 
        id=msg['id'], 
        format='metadata',
        metadataHeaders=['subject', 'from', 'date', 'to', 'cc']
    ).execute()
    
    # Display formatted message
    format_message_display(msg_data)

# Alternative: If you want to process multiple messages and store them
def process_messages(messages, service):
    """Process multiple messages and return formatted data (metadata only)"""
    processed_messages = []
    
    for msg in messages:
        msg_data = service.users().messages().get(
            userId='me', 
            id=msg['id'], 
            format='metadata',
            metadataHeaders=['subject', 'from', 'date', 'to', 'cc']
        ).execute()
        
        headers = msg_data.get('payload', {}).get('headers', [])
        
        # Create clean message object
        clean_msg = {
            'id': msg_data.get('id'),
            'threadId': msg_data.get('threadId'),
            'subject': get_header_value(headers, 'Subject'),
            'from': get_header_value(headers, 'From'),
            'to': get_header_value(headers, 'To'),
            'date': get_header_value(headers, 'Date'),
            'snippet': msg_data.get('snippet'),
            'labels': msg_data.get('labelIds', [])
        }
        
        processed_messages.append(clean_msg)
    
    return processed_messages

# Usage example:
clean_messages = process_messages(messages, service)
for msg in clean_messages:
    pprint.pprint(msg)
    print("\n" + "-"*50 + "\n")

MESSAGE ID: 197d86229aeb9fd5
SUBJECT: 8 Python Libraries So Good, I Stopped Writing My Own Scripts | Abdur Rahman in Python in Plain English
FROM: Medium Daily Digest <noreply@medium.com>
TO: iit2022052@iiita.ac.in
DATE: Sat, 05 Jul 2025 02:20:00 +0000 (UTC)
SNIPPET: Iit Stories for Iit @iit2022052·Become a member Medium daily digest Today&#39;s highlights Abdur Rahman Abdur RahmaninPython in Plain English 8 Python Libraries So Good, I Stopped Writing My Own
--------------------------------------------------------------------------------
BODY:
Stories for Iit
@iit2022052 (https://medium.com/@iit2022052?source=email-a215905353f4-1751668793983-digest.reader-------------------------84100596_a75d_42de_9ad0_29c35bd3384d)
·Become a member (https://medium.com/plans?source=email-a215905353f4-1751668793983-digest.reader-------------------------84100596_a75d_42de_9ad0_29c35bd3384d)

Today's highlights

Abdur Rahman (https://medium.com/@abdur.rahman12?source=email-a215905353f4-1751668793983-diges

In [ ]:
# Gmail API Scope Limitations

The current Gmail authentication is configured with **metadata scope** which allows access to:
- Email headers (subject, from, to, date, etc.)
- Email snippets
- Message IDs and thread IDs
- Labels

## To Access Full Email Content

If you need to read the full email body, you would need to:

1. **Update the authentication scope** to include `https://www.googleapis.com/auth/gmail.readonly` or `https://www.googleapis.com/auth/gmail.modify`
2. **Re-authenticate** to get new credentials with the broader scope
3. **Use `format='full'`** in the API calls

## Current Capabilities

With the metadata scope, you can:
- List emails
- Read email headers and snippets
- Send emails
- Manage labels
- Forward email summaries

This is sufficient for most email management tasks without requiring access to full email content.